In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# googleplaystore.csv: https://drive.google.com/file/d/1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ/view?usp=drive_link
# googleplaystore_user_reviews.csv: https://drive.google.com/file/d/1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO/view?usp=drive_link
downloaded = drive.CreateFile({'id':"1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ"})
downloaded.GetContentFile('googleplaystore.csv')

In [4]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context -> variable que vamos a utilizar
sc = spark.sparkContext

¿Cuál es la aplicación que generó más dinero? (⭐)


In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
rdd = df.rdd
paid_apps_rdd = rdd.filter(lambda x: x['Type'] == 'Paid')

Me quedo con el numero base de descargas (elimino el +) y convierto intalls a int y price a float (remplezando el '$' por '' previamente)


In [6]:
clean_paid_apps_rdd = paid_apps_rdd.map(lambda x: (x['App'], x['Type'], int(x['Installs'].replace('+', '').replace(",", "")), float(x['Price'].replace('$', ''))))
max_revenue_app = clean_paid_apps_rdd.reduce(lambda x, y: x if x[2] * x[3] > y[2] * y[3] else y)
print(f"La aplicacion que mas dinero generó fue: {max_revenue_app[0]}")


La aplicacion que mas dinero generó fue: Minecraft
